# MDA 2021
## HW4-Q3
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on Colab environment.

In [1]:
!apt-get update --fix-missing
!sudo apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 85 not upgraded.


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to processing data

In [4]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Check and extract data
--------------------------------------------------

In [12]:
!ls '/content/drive/My Drive/Sample_Data.zip'

'/content/drive/My Drive/Sample_Data.zip'


In [7]:
!unzip "/content/drive/My Drive/Sample_Data.zip" -d "/content/drive/My Drive/Sampe_Data"

Archive:  /content/drive/My Drive/Sample_Data.zip
replace /content/drive/My Drive/Sampe_Data/Sample_Traffic.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 


the cells above, extract data which is in '/content/drive/My Drive/Sample_Data.zip' 

## Initializing Spark and read data
--------------------------------------------------

In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek

spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [2]:
#part a

import pandas as pd
#first of all, we have to load the data
data  = sc.textFile('/content/drive/My Drive/Sampe_Data/Sample_Traffic.csv')
header = data.first()
data = data.filter(lambda line: line != header)
data = data.sample(False,0.001,101)
#this function is capable of splitting columns of data
#['DEVICE_CODE,SYSTEM_ID,ORIGINE_CAR_KEY,FINAL_CAR_KEY,CHECK_STATUS_KEY,COMPANY_ID,PASS_DAY_TIME']
def parseLine(line):  
  fields = line.split(',')
  DEVICE_CODE = int(fields[0])
  ORIGINE_CAR_KEY = int(fields[2])
  PASS_DAY_TIME = pd.to_datetime(fields[6]).day
  return (ORIGINE_CAR_KEY,PASS_DAY_TIME,DEVICE_CODE)

traffic_rdd = data.map(parseLine)
traffic_rdd.first()
#print (200501,1,10477885) tuple

traffic_rdd = traffic_rdd.map(lambda x:(tuple([x[0],x[1]]),x[2]))
traffic_rdd_gb = sorted(traffic_rdd.groupByKey().mapValues(list).collect())

In [ ]:
traffic_rdd_gb

In [3]:
#part 2
path_baskets = []
all_paths = []
#erecting the baskets of paths
for i in range(len(traffic_rdd_gb)):
  path_baskets.append(traffic_rdd_gb[i][1])
  #merging all the items
  all_paths.extend(path_baskets[i])

In [4]:
import numpy as np
all_paths = list(np.unique(all_paths))
adj_matrix = np.zeros((len(traffic_rdd_gb),len(all_paths)))
for i in range(len(traffic_rdd_gb)):
  for element in list(set(all_paths).intersection(traffic_rdd_gb[i][1])):
    ind = all_paths.index(element)
    adj_matrix[i][ind] = 1

In [5]:
adj_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [5]:
import random
random.seed(101)
i = random.randint(0,len(all_paths))
random_device_code_list = [0] * i + [1]*(len(all_paths) - i)
random.shuffle(random_device_code_list)

In [14]:
from numpy import dot
from numpy.linalg import norm
cos_sim = np.zeros(len(traffic_rdd_gb))
for i in range(len(traffic_rdd_gb)):
  cos_sim[i] = dot(random_device_code_list, list(adj_matrix[i]))/(norm(random_device_code_list)*norm(list(adj_matrix[i])))
cos_sim

array([0.08333333, 0.        , 0.        , ..., 0.        , 0.08333333,
       0.        ])

In [9]:
top_5_idx = list(np.argsort(cos_sim)[-5:])
for i in top_5_idx:
  print(traffic_rdd_gb[i])

((17149337, 1), [100700861, 900205])
((86767658, 1), [900202, 100700868])
((79163174, 1), [900139, 900216])
((19113889, 1), [900216, 100700868])
((23726171, 1), [631355, 100701129])


In [6]:
#part 3
#LSH implementation with cos similarity
import itertools
import collections

In [25]:
def lsh_euclidean(A, b, r, threshold):
  """
  Args:
    A: the (N,D) matrix of vectors to with which to find similar pairs
    b: the number of bands
    r: the number of rows per band
    threshold: a float value [-1,1] determining the required cosine similarity threshold
  Returns:
    a set of pairs with requisite similarity.
  """
  N, D = A.shape
  n = b*r
  
  # Compute signature matrix
  R = A@np.random.randn(D, n)
  S = np.where(R>0, 1, 0)

  # Break into bands
  S = np.split(S, b, axis=1)

  # column vector to convert binary vector to integer e.g. (1,0,1)->5
  binary_column = 2**np.arange(r).reshape(-1,  1)

  # convert each band into a single integer, 
  # convert band matrices to band columns
  S = np.hstack([M@binary_column for M in S])

  # Every value in the matrix represents a hash bucket assignment
  # For every bucket in row i, add index i to that bucket
  d = collections.defaultdict(set)
  with np.nditer(S,flags=['multi_index']) as it:
      for x in it:
          d[int(x)].add(it.multi_index[0])

  # For every bucket, find all pairs. These are the LSH pairs.
  candidate_pairs = set()
  for k,v in d.items():
      if len(v) > 1:
          for pair in itertools.combinations(v, 2):
              candidate_pairs.add(tuple(sorted(pair)))

  # Finally, perform the actually similarity computation
  
  lsh_pairs = set()
  for (i,j) in candidate_pairs:
    if i<441 : 
      if dot(random_device_code_list, list(A[i]))/(norm(random_device_code_list)*norm(list(A[i]))) > threshold:
          lsh_pairs.add(i)
    if j<441 : 
      if dot(random_device_code_list, list(A[j]))/(norm(random_device_code_list)*norm(list(A[j]))) > threshold:
          lsh_pairs.add(j)
  return lsh_pairs

In [ ]:
lsh_euclidean(adj_matrix,8,2,0.1)